# Binäre Klassifikation anhand eines Spam-Filters

## Thema
In dieser Binären Klassifikation handelt es sich um die Vorhersage auf das Eintreffen einer Spam-Email oder einer Ham-Email. Um dies herauszufinden wird ein Datensatz der mehrere Spam-/Ham-Emails mit EmailText enthält verwendet.

## Technische Vorbereitung
Damit alle notwendigen Funktionen und Befehle zur Verfügung stehen, müssen zunächst einige Module mit ihren jeweiligen Klassen importiert werden. Dazu zählt pandas und scikit-learn. Alle Module werden für spätere Datenanalysemethoden genutzt.


In [1]:
import pandas as pd
#from pandas import set_option
from matplotlib import pyplot
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import GridSearchCV

#from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

## Analyse des Datensatzes
Der Datensatz liegt hierbei in einer ".csv" - Datei und beinhaltet 5572 Datensätze. Er ist in zwei Spalten untergliedert. Die Spalte Label beinhaltet die Gruppen Spam und Ham die den jeweiligen Datensatz zugeordnet sind. Bei der zweiten Spalte handelt es sich um den EmailText.


In [2]:
dataframe = pd.read_csv("spam.csv")
print(dataframe.head())
print("")
print(dataframe.describe())


  Label                                          EmailText
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

       Label               EmailText
count   2000                    2000
unique     2                    1925
top      ham  Sorry, I'll call later
freq    1720                      13


Sowohl Label als auch der EmailText besitzen den Datentyp object.

In [3]:
print(dataframe.dtypes)


Label        object
EmailText    object
dtype: object


Durch den folgenden Code wurde die Anzahl von Spam- bzw. Ham-Emails in dem Datensatz gezählt. Dabei fällt auf, dass es sehr unausgeglichen ist und es sich bei den meisten Nachrichten um ham-Emails handelt.

In [4]:
hamCount = dataframe.apply(lambda x: True if x["Label"] == "ham" else False, axis=1)
spamCount = dataframe.apply(lambda x: True if x["Label"] == "spam" else False, axis=1)

print("Anzahl ham  - Emails: ", len(hamCount[hamCount == True].index))
print("Anzahl spam - Emails: ", len(spamCount[spamCount == True].index))


Anzahl ham  - Emails:  1720
Anzahl spam - Emails:  280


## Traings- und Testdaten
Im nächsten Schritt werden die Daten in den Spalten separiert und danach gruppiert. Die Trainingsdaten werden für das Lernen der Muster und Zusammenhänge in den Daten verwendet. Der Algorithmus nutzt diese Daten um daraus zu lernen. Bei den Testdaten handelt es sich um Daten mit der gleichen Wahrscheinlichkeitsverteilung. Diese Daten werden vom Algorithmus vorher nicht genutzt. Mit ihnen kann nachgeweisen werden mit welcher Qualität der Algorithmus auf neue Daten reagieren wird. Die Verteilung verläuft auf 80% Trainingsdaten und 20% Testdaten.

In [5]:
x = dataframe["EmailText"]
y = dataframe["Label"]

x_train,y_train = x[0:4457],y[0:4457]
x_test,y_test = x[4457:],y[4457:]

## Transformieren der Daten
Nachdem die Daten gruppiert wurden müssen die Strings in numerische Werte umgewandelt werden, damit der Klassifikator die Daten nutzen kann. Scikit-Learn stellt hierbei einen "CountVectorizer" zur Verfügung. Er konvertiert eine Sammlung von Text Dokumenten in eine Matrix mit Token-Zahlen. Die Matrix baut sich aus 4457 Zeilen (EmailTexte) und 7756 Spalten (Anzahl unterschiedlicher Wörter) zusammen.

In [6]:
cv = CountVectorizer()  
featuresX = cv.fit_transform(x_train)

print(featuresX.shape)
print(featuresX.toarray())


(2000, 4973)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## Evaluierung der Algorithmen
Im folgenden Code wird mit Hilfe der Genauigkeit der beste Algorithmus herausgefunden. Die Genauigkeit wird dabei anhand des Mittelwertes und der Standardabweichung gemessen. Es stellt sich heraus, dass die logistische Regression und der Support-Vector-Machine-Klassifikator am besten geeignet sind.

In [16]:
models = []
models.append(("LR", LogisticRegression()))
models.append(("NB", GaussianNB()))
models.append(("SVM", SVC()))

results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, shuffle=True)
    cv_results = cross_val_score(model, featuresX.toarray(), y_train, cv=kfold, scoring="accuracy")
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.971000 (0.015297)
NB: 0.913000 (0.023685)
SVM: 0.965000 (0.008660)


## Evaluierung der Algorithmen mit Hilfe der Standardisierung
In diesem Code werden alle Merkmale so transformiert, dass sie einen Mittelwert von 0 und eine Standardabweichung von 1 besitzen. Durch die Verwendung von pipelines wird Datenverlust vermieden. Dabei fällt auf das die Genauigkeit bei allen drei Algorithem gesunken ist. Die logistische Regression und der Support-Vektor-Machine-Klassifikator sind trotzdem noch die bessern.

In [17]:
pipelines = []
pipelines.append(("ScaledLR", Pipeline([("Scaler", StandardScaler()), ("LR", LogisticRegression())])))
pipelines.append(("ScaledNB", Pipeline([("Scaler", StandardScaler()), ("NB", GaussianNB())])))
pipelines.append(("ScaledSVM", Pipeline([("Scaler", StandardScaler()), ("SVM", SVC())])))

results =[]
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=10, shuffle=True)
    cv_results = cross_val_score(model, featuresX.toarray(), y_train, cv=kfold, scoring="accuracy")
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)


ScaledLR: 0.959000 (0.012207)
ScaledNB: 0.900000 (0.016279)
ScaledSVM: 0.911000 (0.018682)


## Verbesserung der Algorithmen
Durch das Verändern/Anpassen von Parametern für einen jeweiligen Algroithmus kann seine Genauigkeit erhöht werde.n

### Verbesserung des Support-Vector-Machine-Klassifikator
Beim SVM lassen sich zwei Parameter anpassen. Zum einen der Kernel und zum anderen der C-Wert. Der Kernel versucht den Trainingsvektor in verschiedene höherdimensionale Räume zu überführen, damit die Vektoren sich leichter trennen lassen. Beim C-Wert wird dem Klassifikator mitgeteilt wie sehr er eine Fehlklassifizierung vermeiden soll. 

In [8]:
scaler = StandardScaler().fit(featuresX.toarray())
rescaledX = scaler.transform(featuresX.toarray())
c_values = [0.1, 0.5, 1.0, 1.5, 2.0]
kernel_values = ["linear", "poly", "rbf", "sigmoid"]
param_grid = dict(C=c_values, kernel=kernel_values)
model = SVC()
kfold = KFold(n_splits=10, shuffle=True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring="accuracy", cv=kfold)
grid_result = grid.fit(rescaledX, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#means = grid_result.cv_results_["mean_test_score"]
#stds = grid_result.cv_results_["std_test_score"]
#params = grid_result.cv_results_["params"]

Best: 0.969000 using {'C': 2.0, 'kernel': 'sigmoid'}


## Klassifikator
Für den Spam-Filter wird der Support-Vector-Machine-Klassifikator genutzt. Damit die einzelnen Klassen einen möglichst breiten Abstand ohne Objekte zueinander besitzen. Des Weiteren müssen auch die Testdaten transformiert werden. 

In [13]:
model = svm.SVC()
model.fit(features,y_train)

features_test = cv.transform(x_test)
print("Die Genauigkeit des Models entspricht:", model.score(features_test,y_test))

Die Genauigkeit des Models entspricht: 0.9856502242152466


In [91]:
tuned_parameters = {'kernel': ['rbf','linear'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]}

model = GridSearchCV(svm.SVC(), tuned_parameters)

model.fit(features,y_train)

print(model.best_params_)

features_test = cv.transform(x_test)
print("Die Genauigkeit des Models entspricht:", model.score(features_test,y_test))

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
Die Genauigkeit des Models entspricht: 0.9874439461883409
